# a. 使用 Transformers 加载量化后的 LLM 大模型（GPTQ & AWQ） 

> 引导文章：[19a. 从加载到对话：使用 Transformers 本地运行量化 LLM 大模型（GPTQ & AWQ）](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/19a.%20从加载到对话：使用%20Transformers%20本地运行量化%20LLM%20大模型（GPTQ%20%26%20AWQ）.md)。

代码文件没有显卡要求，在个人计算机上均可进行对话。

**模型文件约为 4 GB**。

这里还有一个简单的 [🎡 AI Chat 脚本](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/CodePlayground/chat.py)供你尝试，详见：[CodePlayground](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/CodePlayground/README.md#当前的玩具)，点击 `►` 或对应的文本展开。

Llama-cpp-python 关于 GGUF 文件加载的相关链接：[文章 19b](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/19b.%20从加载到对话：使用%20Llama-cpp-python%20本地运行量化%20LLM%20大模型（GGUF）.md) | [代码文件 16b](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Demos/16b.%20使用%20Llama-cpp-python%20加载量化后的%20LLM%20大模型（GGUF）.ipynb)。

在线链接：[Kaggle - a](https://www.kaggle.com/code/aidemos/16a-transformers-llm-gptq) | [Colab - a](https://colab.research.google.com/drive/1cmIDjHriW8aQ5mIsV6ZeTqdnqYe6PoOv?usp=sharing)

## 手动下载模型（推荐）

来试试多线程指定文件下载，对于 Linux，这里给出配置命令，其余系统可以参照[《a. 使用 HFD 加快 Hugging Face 模型和数据集的下载》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/a.%20使用%20HFD%20加快%20Hugging%20Face%20模型和数据集的下载.md)先进行环境配置。你也可以跳过这部分，后面会介绍自动下载。

```bash
sudo apt-get update
sudo apt-get install git git-lfs wget aria2
git lfs install
```

下载并配置 HFD 脚本：

```bash
wget https://huggingface.co/hfd/hfd.sh
chmod a+x hfd.sh
export HF_ENDPOINT=https://hf-mirror.com
```

使用多线程下载指定模型。

### GPTQ

命令遵循 `./hfd.sh <model_path> --tool aria2c -x <线程数>`的格式：

```bash
./hfd.sh neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit --tool aria2c -x 8
```

### AWQ

命令遵循 `./hfd.sh <model_path> --tool aria2c -x <线程数>`的格式：

```python
./hfd.sh solidrust/Mistral-7B-Instruct-v0.3-AWQ --tool aria2c -x 8
```

### GGUF

使用多线程下载指定模型，命令遵循 `./hfd.sh <model_path> --include <file_name> --tool aria2c -x <线程数>`的格式：

```python
./hfd.sh bartowski/Mistral-7B-Instruct-v0.3-GGUF --include Mistral-7B-Instruct-v0.3-Q4_K_M.gguf --tool aria2c -x 8
```

下载完成你应该可以看到类似的输出：

```
Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
145eba|OK  |   6.8MiB/s|./Mistral-7B-Instruct-v0.3-Q4_K_M.gguf

Status Legend:
(OK):download completed.
Downloaded https://huggingface.co/bartowski/Mistral-7B-Instruct-v0.3-GGUF/resolve/main/Mistral-7B-Instruct-v0.3-Q4_K_M.gguf successfully.
Download completed successfully.
```



## Transformer

### 环境配置

In [ ]:
!uv add numpy
!uv add pandas
!uv add transformers
!uv add optimum

### 关于 autogptq 的迁移

需要注意的是，AutoGPTQ 的开发团队在 [24 年 12 月](https://github.com/AutoGPTQ/AutoGPTQ/discussions/758)明确建议所有用户迁移到 GPTQModel。且仓库的 [setup.py](https://github.com/AutoGPTQ/AutoGPTQ/blob/9f7d37072917ab3a7545835f23e808294a542153/setup.py#L11) 文件中包含弃用警告："AutoGPTQ 已停止开发，请过渡到 GPTQModel...计划在近期完全从 HuggingFace 框架中弃用 AutoGPTQ"。

所以这里新增一个小模块进行说明。

在用法方面，既可以使用[官方仓库](https://github.com/ModelCloud/GPTQModel)所叙述的 GPTQModel.load(...)，也可以直接使用 transformers，详见 [GPTQ - Hugging Face](https://huggingface.co/docs/transformers/quantization/gptq?install=GPTQmodel)。

In [ ]:
!uv pip install -v gptqmodel --no-build-isolation  # 安装时间较长
!uv pip install logbar

### GPTQ

#### 导入库

In [ ]:
import os
# 设置模型下载镜像（注意，需要在导入 transformers 等模块前进行设置才能起效）
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import torch
from gptqmodel import GPTQModel
from transformers import AutoTokenizer

下面介绍两种导入模型的方法，实际执行时本地/自动导入二选一。

#### 设置模型路径


In [ ]:
# 如果你已经配置过了，可以直接在 Notebook 中执行下面的命令下载。
# !./hfd.sh neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit --tool aria2c -x 8

如果已经在本地下载了模型，可以通过指定模型路径来加载模型。以下示例假设模型位于当前目录的 `Mistral-7B-Instruct-v0.3-GPTQ-4bit` 文件夹下：

In [ ]:
# # 指定本地模型的路径
# model_path = "./Mistral-7B-Instruct-v0.3-GPTQ-4bit"

如果没有本地模型，设置远程路径（`id` + `/` + `model_name`），导入的时候会自动从 Hugging Face 下载模型：

In [ ]:
# 指定远程模型的路径
model_path = "neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit"

#### 加载模型


In [ ]:
# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 下载并加载模型
model = GPTQModel.load(model_path)

#### 推理测试

In [ ]:
# 输入文本
input_text = "Hello, World!"

# 将输入文本编码为模型可接受的格式（包含attention_mask）
inputs = tokenizer(
    input_text, 
    return_tensors="pt",
    return_attention_mask=True  # 明确返回attention_mask
).to(model.device)

# 生成输出
with torch.no_grad():
    output_ids = model.generate(
        **inputs,  # 传递整个inputs字典，包含input_ids和attention_mask
        max_length=50,
        pad_token_id=tokenizer.pad_token_id
    )

# 解码生成的输出
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 打印生成的文本
print(output_text)

### 关于 autoawq 的迁移

目前 AutoAWQ 已经弃用，可使用另一个项目 vLLM 进行替代：https://github.com/vllm-project/llm-compressor

In [ ]:
uv pip install vllm
# 如果还想安装 autoawq，取消下面的注释
# uv pip install autoawq 'autoawq[kernels]'

### AWQ


#### 导入库

In [ ]:
import os
# 设置模型下载镜像（注意，需要在导入 transformers 等模块前进行设置才能起效）
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import torch
from vllm import LLM, SamplingParams

下面介绍两种导入模型的方法，实际执行时本地/自动导入二选一。

#### 设置模型路径


In [ ]:
# # 如果你已经配置过了，可以直接在 Notebook 中执行下面的命令下载。
# !export HF_ENDPOINT=https://hf-mirror.com
# !./hfd.sh solidrust/Mistral-7B-Instruct-v0.3-AWQ --tool aria2c -x 8

如果已经在本地下载了模型，可以通过指定模型路径来加载模型。以下示例假设模型位于当前目录的 `Mistral-7B-Instruct-v0.3-AWQ` 文件夹下：

In [ ]:
# 指定本地模型的路径
# model_path = "./Mistral-7B-Instruct-v0.3-AWQ"

如果没有本地模型，设置远程路径（`id` + `/` + `model_name`），导入的时候会自动从 Hugging Face 下载模型：


In [ ]:
# 指定远程模型的路径
model_path = "solidrust/Mistral-7B-Instruct-v0.3-AWQ"

#### 加载模型

一些权重不会被加载，对于当前任务来说这是预期的行为。

In [ ]:
# 下载并加载模型（vLLM会自动处理tokenizer）
model = LLM(
    model=model_path,
    quantization="awq",
    tensor_parallel_size=1,
    gpu_memory_utilization=0.9,
)

#### 推理测试

In [ ]:
# 设置生成参数
sampling_params = SamplingParams(
    max_tokens=50,
    temperature=0.8,
    top_p=0.9
)

# 输入文本
input_text = "Hello, World!"

# 生成输出（vLLM自动处理编码和解码）
outputs = model.generate([input_text], sampling_params)

# 打印生成的文本
output_text = outputs[0].outputs[0].text
print(output_text)

### 统一方式加载

“要是所有的模型都能统一就好了，这样就不用查阅其他库的文档了。”

事实上，我们可以选择使用 transformers 进行二者的加载（如果你对参数设置没有更细致的要求）。

#### 下载 autoawq 库（可选）

如果想用 transformers 加载，则需要下载 autoawq 库。

从 `v0.2.7.post3` 开始 awq 会使用 `flash-attn` 库，但部分 GPU 比如在线平台 Kaggle 和 Colab 提供的 T4 不支持，运行后续代码会报错：`RuntimeError: FlashAttention only supports Ampere GPUs or newer.`

故选择下载固定版本：

In [ ]:
!uv pip install autoawq==0.2.7.post2 'autoawq[kernels]==0.2.7.post2'

In [ ]:
import os
# 设置模型下载镜像
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_quantized_model(model_path):
    """统一加载GPTQ或AWQ模型的函数"""
    
    # 加载分词器
    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        trust_remote_code=True
    )
    
    # 修复pad_token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.unk_token or tokenizer.eos_token
    
    # 加载模型 - transformers会自动检测量化格式
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype="auto",
        device_map="auto",
        # attn_implementation="flash_attention_2"  # 可选：使用flash attention
    )
    
    return model, tokenizer

# 测试不同的模型
models = {
    "GPTQ": "neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit",
    "AWQ": "solidrust/Mistral-7B-Instruct-v0.3-AWQ"
}

# 选择要测试的模型
model_type = "GPTQ"  # 或 "AWQ"
model_path = models[model_type]

print(f"加载 {model_type} 模型: {model_path}")

# 加载模型
model, tokenizer = load_quantized_model(model_path)

print(f"模型设备: {model.device}")
print(f"模型数据类型: {model.dtype}")

# 输入文本
input_text = "Hello, World!"

# 将输入文本编码为模型可接受的格式
inputs = tokenizer(
    input_text, 
    return_tensors="pt",
    return_attention_mask=True
).to(model.device)

# 生成输出
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=100,  # max_new_tokens 直接等价于生成的 tokens 数量，参数 max_length = len(input) + max_new_tokens
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id
    )

# 只解码新生成的部分
input_length = inputs['input_ids'].shape[1]
generated_text = tokenizer.decode(output_ids[0][input_length:], skip_special_tokens=True)

print(f"输入: {input_text}")
print(f"生成: {generated_text}")

### 了解提示词模版（prompt_template）

其实非常简单，就是曾经提到的占位符（下图对于 `{{question}}` 的应用）。

![占位符](../Guide/assets/%E5%8D%A0%E4%BD%8D%E7%AC%A6-6055722.png)

举个直观的例子：

In [ ]:
# 定义 Prompt Template
prompt_template = "问：{question}\n答："

# 定义问题
question = "人工智能的未来发展方向是什么？"

# 使用 Prompt Template 生成完整的提示
prompt = prompt_template.format(question=question)
print(prompt)
# print("\n")
# print(f"问：{question}\n答：")

#### tokenizer.chat_template

查看模型的 `chat_template`。

In [ ]:
# 打印 chat_template 信息（如果存在的话）
if hasattr(tokenizer, 'chat_template'):
    print(tokenizer.chat_template)
else:
    print("Tokenizer 没有 'chat_template' 属性。")

### 流式输出

在项目初期认识 API 的时候，文章[《01. 初识 LLM API：环境配置与多轮对话演示》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/01.%20初识%20LLM%20API：环境配置与多轮对话演示.md#流式输出)有提到过流式输出，这是我们一直以来见到的大模型输出方式：逐字（token）打印而非等全部生成完打印。

执行下面的代码试试（无论之前导入的是哪种模型，都可以继续）：

In [ ]:
from transformers import TextStreamer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 创建 TextStreamer 实例
streamer = TextStreamer(
    tokenizer, 
    skip_prompt=True,         # 在输出时跳过输入的提示部分，仅显示生成的文本
    skip_special_tokens=True  # 忽略生成过程中的特殊标记（比如 <pad> / <eos> ...）
)

# 将提示编码为模型输入
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# 设置生成参数
generation_kwargs = {
    "input_ids": input_ids,  # 模型的输入 ID，注意，这不是 Embedding
    "max_length": 200,       # 生成的最大 token 数
    "streamer": streamer,    # 使用 TextStreamer 实现生成过程中逐步输出文本
    "pad_token_id": tokenizer.eos_token_id  # 默认行为，消除 open-end 警告
}

# 开始生成文本
with torch.no_grad():
    # ** 是 Python 中的解包操作符，它将字典中的键值对解包为函数的关键字参数。
    # 在这里，**generation_kwargs 将字典中的参数逐一传递给 model.generate() 方法，
    # 等效于直接写出所有参数：
    # model.generate(input_ids=input_ids, max_length=200, do_sample=True, ...)
    # 你需要注意到，这和之前采用了不同的传参方式，但本质是一样的。
    # 在后续的教程中，会较少地使用这种方式进行传参。
    # 因为这很好的分离了参数，所以也增加了乍一看之下的抽象度，为了初见的直观，将减少使用。
    model.generate(**generation_kwargs)

### 单轮对话


（如果重新启动内核的话，遵循 `导入库`-> `导入模型` -> `当前代码块` 的顺序执行。）

让我们直接设计 `messages`，并应用 `chat_template` 进行对话：


In [ ]:
from transformers import TextStreamer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 定义输入
prompt = input("User: ")

# 定义消息列表
messages = [
    {"role": "user", "content": prompt}
]

# 使用 tokenizer.apply_chat_template() 生成模型输入
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

# 创建 TextStreamer 实例
streamer = TextStreamer(
    tokenizer, 
    skip_prompt=True,         # 在输出时跳过输入的提示部分，仅显示生成的文本
    skip_special_tokens=True  # 忽略生成过程中的特殊标记（比如 <pad> / <eos> ...）
)

# 设置生成参数
generation_kwargs = {
    "input_ids": input_ids,  # 模型的输入 ID，注意，这不是 Embedding
    "max_length": 500,      # 生成的最大 token 数
    "streamer": streamer,    # 使用 TextStreamer 实现生成过程中逐步输出文本
    "pad_token_id": tokenizer.eos_token_id  # 默认行为，消除 open-end 警告
}

# 开始生成文本
with torch.no_grad():
    model.generate(**generation_kwargs)

### 多轮对话

如果重新启动内核的话，遵循 `导入库`-> `导入模型` -> `当前代码块` 的顺序执行。

In [ ]:
from transformers import TextStreamer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 初始化对话历史
messages = []

# 开始多轮对话
while True:
    # 获取输入
    prompt = input("User: ")
    
    # 退出对话条件（当然，你也可以直接终止代码块）
    if prompt.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break
    
    # 将输入添加到对话历史
    messages.append({"role": "user", "content": prompt})
    
    # 使用 tokenizer.apply_chat_template() 生成模型输入
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    
    # 创建 TextStreamer 实例
    streamer = TextStreamer(
        tokenizer, 
        skip_prompt=True,         # 在输出时跳过输入的提示部分，仅显示生成的文本
        skip_special_tokens=True  # 忽略生成过程中的特殊标记（比如 <pad> / <eos> ...）
    )
    
    # 设置生成参数
    generation_kwargs = {
        "input_ids": input_ids,                  # 模型的输入 ID
        "max_length": input_ids.shape[1] + 500,  # 生成的最大 token 数，input_ids.shape[1] 即输入对应的 tokens 数量
        "streamer": streamer,                    # 使用 TextStreamer 实现生成过程中逐步输出文本
        "pad_token_id": tokenizer.eos_token_id   # 默认行为，消除警告
    }
    
    # 开始生成回复
    with torch.no_grad():
        output_ids = model.generate(**generation_kwargs)
    
    # 获取生成的回复文本
    assistant_reply = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
    
    # 将模型的回复添加到对话历史
    messages.append({"role": "assistant", "content": assistant_reply})

注意，这里有一个小坑，你不能简单使用 `output_ids[0]` 来保存回复，因为`output_ids` 中实际上包含了 `input_ids`，打印它们：


In [ ]:
print(tokenizer.decode(input_ids[0], skip_special_tokens=True))
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))